<a href="https://colab.research.google.com/github/Tanish-Sarkar/Elite-Transformers/blob/main/Module4%20-%20Full%20Encoder-Decoder%20Implementation/encoder_block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Adding ScaledDotProductAttention, which is used by MultiHeadAttention
class ScaledDotProductAttention(nn.Module):
    def __init__(self, temperature):
        super().__init__()
        self.temperature = temperature

    def forward(self, q, k, v, mask=None):
        attn = torch.matmul(q / self.temperature, k.transpose(2, 3))

        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)

        attn = F.softmax(attn, dim=-1)
        output = torch.matmul(attn, v)
        return output, attn

In [7]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.d_head = d_model // num_heads

        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        # Using scaled dot-product attention
        self.attention = ScaledDotProductAttention(temperature=self.d_head**0.5)

    def forward(self, x, mask=None):
        B, T, C = x.shape
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)

        Q = Q.view(B, T, self.num_heads, self.d_head).transpose(1, 2)
        K = K.view(B, T, self.num_heads, self.d_head).transpose(1, 2)
        V = V.view(B, T, self.num_heads, self.d_head).transpose(1, 2)

        attn_out, _ = self.attention(Q, K, V, mask)
        attn_out = attn_out.transpose(1, 2).contiguous().view(B, T, C)
        return self.W_o(attn_out)

In [8]:
class TransformerEncoderBlock(nn.Module):
  def __init__(self, d_model=64, num_heads=8, d_ff=256, dropout=0.1):
    super().__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)

    self.ffn = nn.Sequential(
        nn.Linear(d_model, d_ff),
        nn.GELU(),
        nn.Linear(d_ff, d_model),
        nn.Dropout(dropout)
      )

    self.ln1 = nn.LayerNorm(d_model)
    self.ln2 = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, mask=None):
    attn_out = self.mha(x, mask)
    x = self.ln1(x + self.dropout(attn_out))
    ff_out = self.ffn(x)
    x = self.ln2(x + self.dropout(ff_out))
    return x

**Stack multiple blocks**

In [10]:
class TransformerEncoder(nn.Module):
    def __init__(self, num_layers=6, d_model=64, num_heads=8):
        super().__init__()
        self.layers = nn.ModuleList([
            TransformerEncoderBlock(d_model, num_heads) for _ in range(num_layers)
        ])

    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return x

# Test full encoder
full_encoder = TransformerEncoder(num_layers=6)
final_output = full_encoder(x)
print("6-layer encoder output shape:", final_output.shape)

6-layer encoder output shape: torch.Size([4, 16, 64])


In [11]:
d_model = 64
num_heads = 8
batch_size, seq_len = 4, 16
x = torch.randn(batch_size, seq_len, d_model)
encoder_block = TransformerEncoderBlock(d_model=d_model, num_heads=num_heads, d_ff=256)
out = encoder_block(x)
print("Input shape :", x.shape)
print("Output shape:", out.shape)   # should be same as input!
print("All good — shape preserved ✓")

Input shape : torch.Size([4, 16, 64])
Output shape: torch.Size([4, 16, 64])
All good — shape preserved ✓
